# TP SQL Agrégation Informatique 2024

date du révision du document : 23/09/24

Ce TP est effectué en utilisant le SGBD H2.

Rmq: ce TP peut aussi se faire avec un autre SGBD tel que MariaDB.

## Installation

Les fichiers SQL pour créer la base Jeux Olympiques

In [1]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/bd-jo-v2_H2.sql -O bd-jo-v2_H2.sql

--2024-09-30 15:56:49--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/bd-jo-v2_H2.sql
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 382202 (373K) [application/octet-stream]
Enregistre : ‘bd-jo-v2_H2.sql’

bd-jo-v2_H2.sql     100%[===================>] 373,24K  --.-KB/s    ds 0,04s   

2024-09-30 15:56:49 (9,48 MB/s) - ‘bd-jo-v2_H2.sql’ enregistré [382202/382202]



In [2]:
!ls *.sql

bd-jo-v2_H2.sql  bd-jo-v2_mariaDB.sql


Les 8 premières lignes du fichier sql

In [3]:
!head -n 8 bd-jo-v2_H2.sql

-- Pays (codePays, nomP)
-- Sport (sid, nomS)
-- Epreuve (epid, sid*, nomEp, categorie, dateDebut, dateFin)    
-- Athlete (aid, nomAth, prenomAth, dateNaissance, codePays*)
-- Equipe (eqid*, codePays*)
-- AthletesEquipe(eqid*, aid*)
-- RangIndividuel (epid*, aid*, rang)
-- RangEquipe (epid*, eqid*, rang)


### Installer l'adaptateur PostgrSQL / Python

In [4]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# executer uniquemement sur jupyterhub Moodle
# pip install pyspark

**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger les pilote PostgreSQL et H2

https://www.psycopg.org/docs/

In [1]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar

--2024-09-30 16:05:31--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1081604 (1,0M) [application/octet-stream]
Enregistre : ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    ds 0,09s   

2024-09-30 16:05:31 (11,7 MB/s) - ‘postgresql-42.6.0.jar’ enregistré [1081604/1081604]



In [2]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-09-30 16:05:32--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2543011 (2,4M) [application/octet-stream]
Enregistre : ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  8,09MB/s    ds 0,3s    

2024-09-30 16:05:32 (8,09 MB/s) - ‘h2-2.1.214.jar’ enregistré [2543011/2543011]



## Fonctions utiles

## connect_H2

In [6]:
def connect_H2(db,user,password,port=5435):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [7]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount and cursor.description is not None:
            names = [desc[0] for desc in cursor.description]
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr).replace('\n',' ')))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr).replace('\n',' ')[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

## show table

In [8]:
def show_table(connection,table_name):

    query=f"""
SELECT table_name,
       column_name,
       data_type,
       column_default,
       is_nullable,
       character_maximum_length,
       numeric_precision
FROM information_schema.columns 
where lower(table_name)  = '{table_name}'  
"""
    execute(connection,query)
    

## show_schema

In [9]:
def show_schema(connection):

    print('*********** Tables ************')
    query="""
    select TABLE_NAME
    from INFORMATION_SCHEMA.TABLES 
    where TABLE_SCHEMA = 'public'
    """
    execute(connection,query,show=True)

    print('\n\n*********** Domaines ************')
    query="""
    SELECT domain_name,check_clause 
    FROM information_schema.domain_constraints a, information_schema.check_constraints b
    where a.constraint_name=b.constraint_name
    """
    execute(connection,query)

    print('\n\n*********** Attributs ************')
    query=f"""
    SELECT c.table_name,
           c.column_name,
           c.data_type,
           c.column_default,
           c.is_nullable,
           c.character_maximum_length,
           c.numeric_precision
    FROM INFORMATION_SCHEMA.TABLES t, information_schema.columns c
    where t.table_name=c.table_name
    and t.TABLE_SCHEMA = 'public'
    """
    execute(connection,query)
    

## Démarrage du Serveur H2

In [10]:
import psycopg2
import os
import subprocess


local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)

# binaire java : décommenter sur jupyterhub Moodle
# java='/opt/tljh/user/envs/pyspark/bin/java'
java='java'

# chaque utilisateur doit avoir un port différent!
port = 5008

print(f'La commande java est: {java}')
print(f'Le numero du port du serveur H2 est: {port}')
print(f'Les fichiers dans {local_dir}:')
for f in os.listdir(local_dir):
    print(f'   {f}')

La commande java est: java
Le numero du port du serveur H2 est: 5008
Les fichiers dans ./data:


In [2]:
output=subprocess.run(f'{java} --version',shell=True, text=True, stdout=subprocess.PIPE).stdout
print(output)

openjdk 11.0.24 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)



In [3]:
# Défnir le port pour le serveur H2
cmd=f"{java} -Dh2.bindAddress=127.0.0.1  -cp  h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort {port}  -baseDir ./data -ifNotExists"
proc=subprocess.Popen(cmd,
                      shell=True,
                      text=True,
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE, 
                      )

In [11]:
try:
    outs, errs = proc.communicate(timeout=1)
    print(errs)
except :
    print("Serveur H2 en marche")

Serveur H2 en marche


In [12]:
connection =  connect_H2("jo"+f'{port}',"ba","ba",port)

Connection to H2 DB successful


# Requêtes SQL

In [13]:
with open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8') as schemafile:
  execute(connection, schemafile.read())
  connection.commit()

0 rows


## Exemple

Tous les athlètes

In [14]:
cursor=execute(connection,query="SELECT * FROM athlete",show=True)
# préciser le parametre top=n pour afficher n lignes
# cursor=execute(connection,query="SELECT * FROM athlete",show=True, top=20)

2431 rows
|aid |nomath                 |prenomath        |datenaissance|codepays|
|----------------------------------------------------------------------
|1   |BJOERNDALEN            |Ole Einar        |1974-01-27   |NOR     |
|2   |BJOERGEN               |Marit            |1980-03-21   |NOR     |
|3   |AN                     |Victor           |1985-11-23   |RUS     |
|4   |PECHSTEIN              |Claudia          |1972-02-22   |GER     |
|5   |WÜST                   |Ireen            |1986-04-01   |NED     |
|6   |SVENDSEN               |Emil Hegle       |1985-07-12   |NOR     |
|7   |AMMANN                 |Simon            |1981-06-25   |SUI     |
|8   |KRAMER                 |Sven             |1986-04-23   |NED     |
|9   |SABLIKOVA              |Martina          |1987-05-27   |CZE     |
|10  |HAMELIN                |Charles          |1984-04-14   |CAN     |
|11  |MORGENSTERN            |Thomas           |1986-10-30   |AUT     |
|12  |HELLNER                |Marcus           |1985-1

In [15]:
cursor.query

b'SELECT * FROM athlete'

In [17]:
cursor.rowcount

2431

In [18]:
cursor.description

(Column(name='aid', type_code=23),
 Column(name='nomath', type_code=1043),
 Column(name='prenomath', type_code=1043),
 Column(name='datenaissance', type_code=1082),
 Column(name='codepays', type_code=1043))

## TD 5 SQL JOINTURES

### Jointures

#### Q1. Les athlètes français (nom pays = 'France') (104 lignes).

In [19]:
query="""
select a.nomAth, a.prenomAth
from Pays p, Athlete a
where a.codePays = p.codePays and p.nomP='France'
"""

cursor=execute(connection,query,show=True)

104 rows
|nomath                 |prenomath     |
|---------------------------------------
|FOURCADE               |Martin        |
|LAMY CHAPPUIS          |Jason         |
|VAULTIER               |Pierre        |
|CHAPUIS                |Jean Frederic |
|BRUNET                 |Marie Laure   |
|DORIN HABERT           |Marie         |
|ANTHONIOZ              |Deborah       |
|MISSILLIER             |Steve         |
|BOVOLENTA              |Arnaud        |
|MARTINOD               |Marie         |
|GAILLARD               |Jean Marc     |
|DE LE RUE              |Paul-Henri    |
|PINTURAULT             |Alexis        |
|BEATRIX                |Jean Guillaume|
|DUVILLARD              |Robin         |
|MANIFICAT              |Maurice       |
|PERRILLAT BOITEUX      |Ivan          |
|JOSSERAND              |Marion        |
|RAMOIN                 |Tony          |
|TRESPEUCH              |Chloe         |
|MATTEL                 |Coline        |
|MIDOL                  |Jonathan      |
|ROLLAN

In [ ]:
query="""
select nomAth, prenomAth
from Pays natural join Athlete
where nomP='France'
"""

cursor=execute(connection,query,show=True)

#### Q2. Les épreuves (sport, nom d'épreuve, catégorie) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (98 lignes).

In [ ]:
query="""
select distinct e.nomep, e.categorie, s.nomsp
from epreuve e, sport s
where e.sid = s.sid
order by s.nomsp desc, e.nomep desc
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select nomep, categorie, nomsp
from epreuve natural join sport
order by nomsp desc, nomep desc
"""

cursor=execute(connection,query,show=True)

#### Q3. Les athlètes ayant participé à une équipe de France (30 lignes).

In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

#### Q4. Les épreuves (sport, épreuve, catégorie) auxquelles participent des équipes (25 lignes).

In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

#### Q5. Le pays qui a gagné, en équipe, la médaille d'or dans l'épreuve de la catégorie 'Femmes' intitulée 'relais 4x6km' du sport 'Biathlon' (Ukraine).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q6. Les athlètes ayant participé à au moins une épreuve individuelle et au moins une par équipe (372 lignes).

In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

#### Q7. Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

Q8. Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

Q9. Les athlètes ayant gagné une médaille dans une épreuve individuelle, mais ayant été disqualifiés
dans une autre (14 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

Q10. Les athlètes ayant gagné (au moins) une médaille dans une épreuve individuelle (182 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

Q11.  Les noms de pays qui n’ont qu’une seule équipe (9 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

Q12. Les athlètes ayant participé à exactement 2 épreuves individuelles (402 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

## TME5 SQL JOINTURES / EXISTS

### Jointures

#### Q1. Les sports auxquels LESSER Erik a participé en supposant qu'il a participé aux épreuves
individuelles (2 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q2. Les athlètes ayant participé aux épreuves individuelles de (au moins) deux sports différents (2 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q3. Les dates de début et de fin des Jeux Olympique 2014.

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q4. Les pays qui ont des participants de moins de 18 ans ou de plus que 40 au 24-02-2016
(Attention : il faut éliminer les athlètes dont on ne connaît pas la date de naissance) (9 ou 7 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

### NOT EXISTS / NOT IN

#### Q5. Les équipes n'ayant pas d'athlète (la base de données ne contient pas l'information sur les participants) (13 lignes)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q6. Les équipes qui ont exactement 2 athlètes. Retourner l'équipe en question avec ses deux seuls athlètes (114 lignes).


In [ ]:
query="""



"""

cursor=execute(connection,query,show=True)

#### Q7. Les sports qui n'ont pas d'épreuves de catégorie Mixte (12 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

####  Q8. Les athlètes qui ont gagné une médaille d’or (au moins) mais pas de médaille d’argent ni de bronze (53 lignes)

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

#### Q9. Les athlète(s) qui ont fini dernier d’une épreuve individuelle. Indiquer leur nom, prenom, le nom
du sport et de l’épreuve et leur rang. Attention : il faut filtrer les athlètes disqualifiés (73 lignes).

In [ ]:
query="""


"""

cursor=execute(connection,query,show=True)

## TD6 REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY

### Q1. Les athlètes ayant participé à (au moins) une épreuve individuelle et (au moins) une épreuve
par équipe. Exprimer la requête en utilisant ANY (au lieu de IN; voir requête 2.6 dans TD5).
Résultat : (372 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Q2.

#### a) Les pays ayant au moins un athlète (requête équivalente: les pays ayant participé aux JO).
Résultat : (88 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### b) Les pays ayant exactement un seul athlète
Résultat : (18 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)


### Q3

#### a) Les athlètes qui n'ont jamais été disqualifiés aux épreuves individuelles.
Résultat : (2194 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### b) Les pays qui n'ont pas eu d'athlète disqualifié aux épreuves individuelles.
Résultat : (143 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### c) Les pays qui ont participé aux JO et qui n'ont pas eu d'athlètes disqualifiés aux épreuves
individuelles.
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### d) Les pays qui ont participé aux JO et qui n'ont pas eu des athlètes disqualifiés ni en
individuel ni par équipe
Résultat : (25 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Q4

#### a) Les athlètes n'ayant pas gagné de médaille ni en individuel ni en équipe
Résultat : (1921 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### b) Les pays ayant participé aux JO et n'ayant pas gagné de médaille ni en individuel ni en équipe.
Résultat : (63 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)


### Q5. L'(es) épreuve(s) avec la plus grande durée.
Résultat : Hockey sur glace, Hockey sur glace, Hommes, 16 jours

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Q6. Les athlètes ayant gagné une médaille à toutes les épreuve individuelles auxquelles ils ont
participé. Résultat : (109 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## TME6 REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY

### ANY / ALL / IN / EXISTS

Exprimer les requêtes suivantes en SQL :

#### Q1. Les sports dont toutes les épreuves ont duré un seul jour.
Résultat : Ski de fond, Ski alpin, Biathlon

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q2. Les sports qui n'ont pas d'épreuves de categorie Mixte.
Résultat : (12 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q3. Les équipes dont aucun athlète n'a gagné de médaille aux épreuves individuelles.
Attention : il y a des équipes sans athlètes.
Résultat : (265 lignes avec les équipes sans athlètes - 252 lignes sans les équipes sans
athlètes)

Avec les équipes sans athlètes :

In [ ]:
query="""


"""
execute(connection,query,show=True)

Sans les équipes sans athlètes :

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q4. La nationalité de l'athlète le/la plus jeune. Attention : il y a des athlètes dont on ne connaît pas la date de naissance.
Résultat : ('29/11/1998','JPN')

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q5. Le plus jeune athlète de chaque pays.
Résultat : (26 lignes)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## TME7 REQUÊTES D'AGRÉGATION ET DIVISION

On reprend le schéma schéma « Jeux Olympiques d'hiver 2014 ».

PAYS (CODEPAYS, NOMP)

SPORT (SID , NOM SP)

EPREUVE (EPID, SID*, NOMEP, CATEGORIE, DATEDEBUT, DATEFIN)

ATHLETE (AID, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)

EQUIPE (EQID, CODE PAYS*)

ATHLETESEQUIPE (EQID*, AID*)

RANGINDIVIDUEL (EPID*, AID*, RANG)

RANGEQUIPE (EPID*, EQID*, RANG)

### Fonctions d’agrégation « COUNT, SUM, AVG, MIN, MAX »

#### Q1. Le nombre d’athlètes.
Résultat (1 ligne) : 2431

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q2. Le nombre d’athlètes ayant participé à au moins une épreuve en individuel.
Résultat (1 ligne) : 1558

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q3. L'âge moyen des sportifs dont le code pays est 'FRA' (France) au 06/02/2014.
Résultat (1 ligne) : 26,8
Aide :
– utilisez round(valeur,nb) pour garder seulement nb décimales à valeur
– sous Oracle, utilisez: to_date('06/02/2014','dd/mm/YYYY')

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q4. La durée moyenne, minimale et maximale des épreuves.
Résultat (1 ligne) : « Durée moyenne = 1,98 min = 1 max = 16 »
Aide : utilisez l’opérateur de concaténation ||
Attention : entre le 10/01/2014 et le 13/01/2014, il y a une durée de 4 jours (et non pas 3).

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q5. Le nombre moyen d'athlètes par pays, c'est-à-dire le nombre d'athlètes divisé par le nombre
de pays (ayant au moins un athlète). Résultat (1 ligne) : 27,625

In [ ]:
query="""


"""
execute(connection,query,show=True)

Partitionnement « group by »

#### Q6. Pour chaque pays, le nom du pays et le nombre d’athlètes, ordonner par nombre d’athlètes
croissant.
Résultats (88 lignes) : (PAK,1) ; (HKG, 1) ; ... ; (USA, 196) ; (CAN,221)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q7. Le nombre moyen d'athlètes par pays (avec group by). Aide : compter le nombre
d’athlètes dans chaque pays (ayant au moins un athlète), puis faire la moyenne.
Résultat (1 ligne) : 27,625

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q8. Pour chaque équipe, l’eqid de l'équipe et le nombre d'athlètes, ordonner par nombre
d’athlètes décroissant.
Résultats (296 lignes) : (164,25) ; (165,25) ; (166,25) ; ... ; (180,2) ; (181, 2) ; (182, 2)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q9. Pour chaque catégorie, la catégorie et le nombre d'épreuves.
Résultats (3 lignes) : (Femmes,43) ; (Mixte,6) ; (Hommes,49)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q10. Pour chaque sport, le nom du sport et le nombre d'épreuves, ordonner par nombre d'épreuves
décroissant.

Résultats (15 lignes) : (Patinage de vitesse,12) ; (Ski de fond,12) ; ... ;(Hockey sur glace,2)

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q11. Pour chaque pays, le code du pays, le nombre de médailles en épreuve individuelle gagnées et le nombre d'athlètes ayant gagnés au moins une médaille. Ordonner par nombre de médailles décroissant.
Aide : 2 tables seulement sont nécessaires.
Résultats (24 lignes) : (NOR, 24,19) ; (NED,22,15) ; ...

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q12. Pour chaque pays et sport, le code du pays, le sid du sport, le nombre de médailles en épreuve individuelle gagnées, le nombre d'athlètes ayant gagnés au moins une médaille, ordonner d'abord par code pays, puis par nombre de médailles décroissant.
Résultats (84 lignes) : (AUS,12,2,2); (AUS,15,1,1);(AUT,13,9,7);(AUT,15,2,2);...

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Partitionnement avec « group by / having »

#### Q13. L’eqid de la ou des équipes qui sont composées :
##### a) d'exactement 10 athlètes. Résultat (1 ligne) : 226

In [ ]:
query="""


"""
execute(connection,query,show=True)

##### b) du plus d’athlètes pour ces JO.

Résultats (3 lignes) : 164 ; 165 ; 166

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q14. Le nombre d'épreuves en individuel où il y a eu au moins 100 participants.
Résultat (1 ligne ) : 2

In [ ]:
query="""


"""
execute(connection,query,show=True)

#### Q15. Le nom des pays qui ont gagné au moins 20 médailles aux épreuves individuelles.
Résultats (3 lignes) : Pays-Bas ; États-Unis ; Norvège

In [ ]:
query="""


"""
execute(connection,query,show=True)

### Division en SQL

#### Q16. Le sid des sports qui ont des épreuves dans toutes les catégories existantes.
Résultats (3 lignes) : 1 ; 6 ; 7

Principe : pour chaque épreuve, on compte le nombre de catégories, puis on regarde si il est
égal au nombre total de catégories d’épreuves.

In [ ]:
query="""


"""
execute(connection,query,show=True)

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Q17. Le nom des pays qui ont participé aux épreuves en individuel de tous les sports en individuel.

Résultats (3 lignes) : (Russie,12) ; (États-Unis,12) ; (Italie,12)

In [ ]:
query="""


"""
execute(connection,query,show=True)

## Requêtes avec paramètres

In [ ]:
cursor=connection.cursor()
cursor.execute("SELECT * FROM athlete WHERE aid=(%s)", '1')
print(cursor.fetchall())


# Manipulation de données : SQL DDL et DML

Les exercices suivantes correspondent aux chapitres correspondants dans les documents
*poly-TD-etudiants-2023.pdf* et *poly-TME-etudiants-2023.pdf*
sur Moodle.

## TD8 CREATION DE SCHEMAS- CONTRAINTES D'INTEGRITE

On considère le schéma Entreprise décrit ci-dessous.

EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)

PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)

EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)

GRILLE_SAL (Profil, salaire)

La clé primaire de chaque relation est soulignée et les attributs des clés étrangères sont suivis d’un
astérisque. Cette base contient des informations sur des employés et sur les projets dans lesquels ils
sont impliqués. DateEnreg dans la table employe donne la date à laquelle l'employé a été enregistré
dans la BD. Ces employés sont embauchés dans un projet sur un profil donné et perçoivent un
salaire en fonction de ce profil. Le chef d'un employé dans la table Employé et le chef d'un projet
dans la table Projet sont des employés. En plus des contraintes de clé et contraintes référentielles
indiquées dans le schéma, on voudrait intégrer les contraintes suivantes :

* Domaines
    - Le numéro de sécurité sociale possède exactement 5 chiffres.
    - Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils peuvent en avoir moins).
    - La ville d’un employé (VillE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et sa longueur ne dépasse pas 9 caractères.
    - Le numéro d’un projet varie entre 5 et 7 chiffres.
    - Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.
    - Le budget est un entier sur 6 chiffres (sans virgule)
    - Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table Employé.

* Clés
    - Il n'y a pas deux employés avec le même nom et le même prénom.

* Not-null
    - Chaque projet doit avoir un responsable.
    - Dans la table Embauche, NumSS; NumProj et Profil représentent un employé, un projet et un profil existant.

### Donner en SQL les instructions de création des contraintes de domaines indiquées dans l'énoncé.

In [ ]:
execute(connection,"drop all objects")

In [ ]:
query="""
/* Le numéro de sécurité sociale possède exactement 5 chiffres.*/
drop domain dnumss if exists;
create domain dnumss as  varchar(5)
"""
execute(connection,query)

In [ ]:
query="""
/* Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils
peuvent en avoir moins).*/
drop domain dchaines if exists;
create domain dchaines as  varchar(20);
"""
execute(connection,query)

In [ ]:
query="""
/* La ville d’un employé (VilleE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et
sa longueur ne dépasse pas 9 caractères.*/
drop domain dvilles if exists;
create domain dvilles as  ...
"""
execute(connection,query)

In [ ]:
query="""
/*Le numéro d’un projet varie entre 5 et 7 chiffres.*/
drop domain dnumproj if exists;
create domain dnumproj as ...

/* Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.*/
drop domain dsal if exists;
create domain dsal as

/*Budget sur 6 chiffres*/
drop domain dbudget if exists;
create domain dbudget as  ...

/*Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table
Employé.*/
drop domain ddatenaiss if exists;
create domain ddatenaiss as ...
"""
execute(connection,query)


### Donner en SQL les instructions de création du schéma de la base en leur associant les contraintes de clés, référentielles et de domaines indiquées dans l'énoncé.

In [ ]:
query="""
/*EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)*/
create table Employe(




)
"""
execute(connection,query)


In [ ]:
query="""
/*PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)*/
create table Projet(


)
"""
execute(connection,query)

In [ ]:
query="""
/*GRILLE_SAL (Profil, salaire)*/
create table Grille_sal(



)
"""
execute(connection,query)


In [ ]:
query="""
/*EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)*/
create table Embauche(



)
"""
execute(connection,query)


3. Un responsable de projet doit habiter la ville du projet dont il est responsable. Donner en
SQL l'instruction permettant d’exprimer cette contrainte en considérant que la table projet a été créée/ Utiliser la commande

      alter table <table> add constraint <nom><expression>

In [ ]:
query="""
alter table PROJET add constraint resprojet check
"""
execute(connection,query)


Commande utile:
    

In [ ]:
query="""
alter table PROJET drop constraint resprojet
"""
execute(connection,query)


## TME8 CREATION DE SCHEMAS- CONTRAINTES D'INTEGRITE

Dans ce qui suit, il vous est demandé d'effectuer des insertions de n-uplets dans des tables.

Syntaxe des insertions :

    insert into Table(attr1, attr2, ...) values ('val1', 'val2', ....)
    insert into Table values ('val1', 'val2', ....)
    insert into Table <requête>
    

Syntaxe des suppressions :

    delete from table
    delete from table where <cond>

Syntaxe des mise à jour :

    upate table set attr = valeur where <cond>
    upate table set attr = <requete> where <cond>

In [ ]:
connection = connect_H2(f"employes{port}","ba","ba",port)

### Insérez dans chaque table au moins un n-uplet qui vérifie les contraintes d'intégrité.

In [ ]:
query="""
insert into employe(NUMSS,NOME,PRENOME, VILLEE , DATENAISS ) values (12345, 'Smith', 'John', 'Paris', '1975-08-25' ))
insert into projet ...
insert into embauche ...
insert int grille_sal ...
"""
execute(connection,query)

### Proposez des insertions qui violent les contraintes d'intégrité définies pour chaque table.

#### Proposer une insertion dans la table Employé qui ne respecte pas la contrainte de clé primaire.

In [ ]:
query="""
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
   values (21456, 'DUPONT', 'Marc', 'paris', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ;
insert into employe
"""
execute(connection,query)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de limite d'âge.

In [ ]:
query="""
insert into employe


"""
execute(connection,query)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de longueur de
l'attribut NumSS.

In [ ]:
query="""
insert into employe
"""
execute(connection,query)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte sur les villes possibles.

In [ ]:
query="""
insert into employe
"""
execute(connection,query)

#### Proposer une insertion dans la table Grille_SAL qui ne respecte pas la contrainte sur le salaire qui ne doit pas dépasser 90 000.

In [ ]:
query="""
insert into  Grille_sal ....
"""
execute(connection,query)

#### Proposer une insertion dans la table Projet qui ne respecte pas la contrainte référentielle vers Employe.

In [ ]:
query="""
insert into Projet values
"""
cursor=execute(connection,query)

#### Proposer une insertion dans la table Embauche qui ne respecte pas une des contraintes
référentielles.

In [ ]:
query="""
insert into employe ...

"""
execute(connection,query)

## TD9 CREATION DES SCHEMAS - MODIFICATION DES DONNEES.

### Villes et Pays

On veut créer un schéma relationnel pour stocker des informations sur des villes et des pays.

#### 1. Traduisez le schéma relationnel suivant en instructions SQL:

    Ville(nom, population, pays*)
    Pays(nom, capitale*)

où pays est une référence vers un pays dans la table Pays et capitale est une référence vers une ville dans la table Ville.

In [ ]:
connection = connect_H2(f"villes{port}","ba","ba",port)

In [ ]:
query="""
create table Ville(



);
create table Pays(



);




"""
execute(connection,query)


#### Insérez la France avec sa capitale Paris (3 millions d'habitants) dans la base de données.

In [ ]:
query="""




"""
cursor=execute(connection,query)

#### Modifiez le schéma de telle manière que la suppression d'un pays déclenche automatiquement la suppression de toutes les villes du pays.
                                             

In [ ]:
query="""
alter table Ville drop constraint fk_pays;
alter table Ville add constraint fk_pays ....

alter table Pays drop constraint fk_pays_capitale;
alter table Pays add constraint fk_pays_capitale .....
"""
execute(connection,query)



#### Effacez les deux tables Ville et Pays du schéma.

In [ ]:
query="""




"""
execute(connection,query)


### Arbres ordonnées

Voici une table Arbre qui permet de stocker des arbres ordonnés dans une base de données
relationnelle:

In [ ]:
connection = connect_H2(f"arbres{port}","ba","ba",port)

In [ ]:
query="""
drop table Arbre if exists;
create table Arbre(
    id numeric(10),
    par numeric(10),
    pos numeric(2),
    constraint pk primary key (id),
    constraint fk foreign key (par) references Arbre (id));
"""
execute(connection,query)


- id est l'identifiant du nœud,
- par est l'identifiant du parent,
- pos est la position parmi les enfants ;

#### Inserez l'arbre binaire (1(2(4,5),3(6,7(8,9))))) dans la base de données.

In [ ]:
query="""

insert into Arbre


"""
execute(connection,query,show=True)


#### Comment faut il modifier le schéma pour empêcher qu'un noeud a deux enfants à la même position?

In [ ]:
query="""
alter table Arbre add constraint ....
"""
execute(connection,query)



#### Effacez le sous-arbre 3(6,7(8,9)) de la base de données.

In [ ]:
query="""



"""
execute(connection,query)


#### Est-ce que l'instruction suivante est possible sur l'arbre initial (avant l'effacement) ?

In [ ]:
query="""
delete from Arbre where id=3
"""
execute(connection,query)

#### Comment peut-on modifier le schéma pour effacer le sous-arbre avec l'instruction précédente ?

In [ ]:
query="""



"""
execute(connection,query)

#### Est-ce qu'il est possible de modifier l'identifiant d'un nœud ?

#### Comment peut-on modifier le schéma pour permettre l'instruction précédente ?

In [ ]:
query="""



"""
execute(connection,query,show=True)

#### Quel est le résultat des instructions suivantes (sur l'arbre initial):

In [ ]:
query="""
alter table Arbre drop constraint fk ;
alter table Arbre add constraint fk foreign key (par) references Arbre(id)
      on delete set null;
delete from Arbre where id=3;
select * from Arbre
"""
execute(connection,query,show=True)

# Transactions

In [ ]:
connection.commit()

In [ ]:
resultat=execute(connection, "SELECT * FROM athlete where aid<12",True)

In [ ]:
delete_comment = "DELETE FROM athlete where aid<10"
execute(connection, delete_comment)

In [ ]:
execute(connection, "SELECT * FROM athlete where aid < 12",True)

In [ ]:
connection.rollback()

In [ ]:
execute(connection, "SELECT * FROM athlete where aid < 12",True)

# Fermer la connexion

In [ ]:
connection.commit() # implicit avec close
connection.close()